# Linux私房手册-常见场景问题收集

所有测试，代码使用的环境是CentOS7.0，不同版本的Linux可能会略有不同。

## Linux常用Shell命令

### `find`命令

#### 讲解

讲解比较全面，但是不够细致：
- [find常用参数详解](https://www.cnblogs.com/yinzhengjie/p/6207318.html)

主要讲解查找文件的5种方式，例子较多：
- [Linux中find命令用法大全](https://www.cnblogs.com/Ido-911/p/9638612.html)

#### 例子

1. 搜索名字为`index.html`的文件并且改名为`index.html.2019-09-23`的样式:
```
find /etc/ /tmp/ /root/ -name "index.html" -exec mv {} {}.`date +%F` \;
```
2. 搜索30天以前的文件并且删除：
```
find /tmp/ -atime +30 -exec rm -rf {} \;
```

其中`{}`表示搜索出来的结果，`\;`不知道具体的意思，但是需要加在命令的后面。

### `date`命令

#### 讲解

`date`命令用来显示系统时间，后面可以加参数进行格式化：
- [Linux下date命令](https://blog.csdn.net/jk110333/article/details/8590746)

#### 例子

1. `date "+%Y-%m-%d"`，输出`2013-02-19`。
2. `date "+Y-%-m-%d"`，输出`2013-2-19`，其中`%-m`表示月份前面不补0。
3. `date -d tomorrow`，输出`2019-9-23`。

### `touch`命令

#### 讲解

`touch`命令用于创建空白文件，并且设置文件的时间：
- [touch命令](https://www.cnblogs.com/joe-four/p/cnblogs_joe_four_06.html)

#### 例子

1. 修改文件test.txt的时间和anaconda-ks.cfg时间相同：
```
touch -r  test.txt  /home/anaconda-ks.cfg
```
2. 修改 anaconda-ks.cfg的读取修改时间为"2020-01-01 03:00:01"：
```
touch -d "2020-01-01 03:00:01" anaconda-ks.cfg
```

## 配置服务器`sftp`服务

### 说明 

[Linux（CentOS）上配置 SFTP](https://www.cnblogs.com/whatmiss/p/7068772.html)

### 注意事项

这里有几个坑需要注意：
1. 网上几乎所有的教程都会说要向下面这样修改`sshd_confg`配置文件：
```
Match User sftpuser #这行用来匹配用户
ChrootDirectory /datas/www  #用chroot将用户的根目录指定到/datas/www ，这样用户就只能在/datas/www下活动
AllowTcpForwarding no
ForceCommand internal-sftp #指定sftp命令
```

但是，在`Red Hat Enterprise Linux Server release 7.0`版本下，`Match User sftpuser`后面的几行要缩进一个`tab`制表符，不然不生效。
2. 不能连接，总是有如下的错误：
```
> Write failed: Broken pipe     
> Couldn't read packet: Connection reset by peer
```
这个是因为从设置的用户的`ftp`根目录向上一直到系统根目录，所有的文件夹的目录拥有者必须是`root`，组可以不是`root`，且所有文件夹的权限必须是755或者750，即不可以具有群组写入权限。如果要有写权限怎么办？目前能想到的是在用户根目录下新建其它用户的目录，并赋予写权限，但此时根目录仍然没有写权限，还没有查到相应得解决方案。

## 配置`Crontab`定时计划

### 说明

[Linux-Centos用crontab定时运行python脚本详细步骤](https://www.cnblogs.com/qq874455953/p/9589207.html)

### 注意事项 

1. `crontab`默认的路径是当前用户的`home`目录，因此如果脚本不在当前用户的`home`目录的话，不管是`crontab`命令本身，还是要执行的脚本里面的路径，都需要使用绝对路径。比如，脚本经常会这样写：
```python
f.open("test.txt", 'w')
f.close()
```
表示在脚本的当前目录下打开一个"test.txt"的文件，此时很可能失效，因为系统会在当前登录用户的`home`目录下生成"test.txt"，而不是在和脚本文件相同的目录下。